In [2]:
from sparseautomodel import SparseAutoModelForCausalLM
import torch

model_path = "/home/paperspace/zephyr-training/data/llama-7b-chat-50sparse-int8"
model = SparseAutoModelForCausalLM.from_pretrained(
    model_path, 
    device_map="cpu",
    torch_dtype=torch.float16,
    trust_remote_code=True,
)

('/home/paperspace/zephyr-training/data/llama-7b-chat-50sparse-int8',)


KeyboardInterrupt: 

In [2]:
def generate_prompts(tokenizer, dataset, messages_column_name="messages", num_prompts=1):
    messages = dataset.shuffle().select(range(num_prompts))[messages_column_name]
    system_prompt = {
        "content": "",
        "role": "system"
    }
    prompts = []
    for message in messages:
        convo = [system_prompt] + message[:-1]
        prompt = tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=True)
        prompts.append(prompt)
    return prompts

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/home/paperspace/zephyr-training/data/llama-7b-chat-50sparse-int8"
dataset_path = "HuggingFaceH4/ultrachat_200k"

model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_path)

/home/rshaw/zephyr-training/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.83s/it]


In [5]:
!pwd

/home/rshaw/zephyr-training/evaluation


In [17]:
import torch
def print_layer_sparsity(model: torch.nn.Module) -> None:
    for name, module in model.named_children():
        if isinstance(module, torch.nn.Linear):
            mask = torch.where(module.weight == 0, torch.tensor(0, dtype=torch.uint8), torch.tensor(1, dtype=torch.uint8))
            print(f"[Layer {name} sparsity = {torch.sum(mask == 0)/mask.numel()}]")
        else:
            print_layer_sparsity(module)

In [19]:
print_layer_sparsity(model)

[Layer q_proj sparsity = 0.5000032782554626]
[Layer k_proj sparsity = 0.5000081062316895]
[Layer v_proj sparsity = 0.5000081062316895]
[Layer o_proj sparsity = 0.5000024437904358]
[Layer gate_proj sparsity = 0.5000011324882507]
[Layer up_proj sparsity = 0.500001072883606]
[Layer down_proj sparsity = 0.5000020265579224]
[Layer q_proj sparsity = 0.5000019073486328]
[Layer k_proj sparsity = 0.5000078678131104]
[Layer v_proj sparsity = 0.5000078678131104]
[Layer o_proj sparsity = 0.5000019669532776]
[Layer gate_proj sparsity = 0.5000009536743164]
[Layer up_proj sparsity = 0.5000007748603821]
[Layer down_proj sparsity = 0.5000020861625671]
[Layer q_proj sparsity = 0.500002384185791]
[Layer k_proj sparsity = 0.5000083446502686]
[Layer v_proj sparsity = 0.5000076293945312]
[Layer o_proj sparsity = 0.5000024437904358]
[Layer gate_proj sparsity = 0.5000009536743164]
[Layer up_proj sparsity = 0.500001072883606]
[Layer down_proj sparsity = 0.5000019073486328]
[Layer q_proj sparsity = 0.5000022053

[Layer lm_head sparsity = 5.882263394596521e-06]


In [20]:
print(tokenizer.chat_template)

{% for message in messages %}
{% if message['role'] == 'user' %}
{{ '<|user|>
' + message['content'] + eos_token }}
{% elif message['role'] == 'system' %}
{{ '<|system|>
' + message['content'] + eos_token }}
{% elif message['role'] == 'assistant' %}
{{ '<|assistant|>
'  + message['content'] + eos_token }}
{% endif %}
{% if loop.last and add_generation_prompt %}
{{ '<|assistant|>' }}
{% endif %}
{% endfor %}


In [21]:
from datasets import load_dataset

dataset = load_dataset(dataset_path, split="train_sft")

In [22]:
tokenizer.pad_token_id = tokenizer.eos_token_id 

In [23]:
import time

prompts = generate_prompts(tokenizer, dataset, num_prompts=1)

start = time.perf_counter()
for prompt in prompts:
    inps = tokenizer(prompt, return_tensors="pt")
    for key in inps:
        inps[key] = inps[key].to("cuda")
    
    outputs_tokens = model.generate(**inps, max_new_tokens=1000)

    print(tokenizer.batch_decode(outputs_tokens)[0])
    print("\n\n\n")
torch.cuda.synchronize()
end = time.perf_counter()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> <|system|>
You are a helpful chatbot</s> 
<|user|>
Here is a piece of text: Tour of the crop trials at the AHDB's Strategic Potato (SPot) Farm initiative at the Elveden Estate. Pictured: Agronomist Graham Tomalin explains the herbicide trials. Picture: Chris Hill.
East Anglia’s potato-growing knowhow is being coupled with the region’s scientific expertise in a bid to improve crop performance across the sector.
Visitors were invited to a farm walk at the Elveden Estate to discuss the progress of crop trials in the third year of the Strategic Potato (SPot) Farm initiative, co-ordinated by the Agriculture and Horticulture Development Board’s potatoes division (AHDB Potatoes).
They were told that while academic research and anecdotal evidence from farmers can both give valuable insights – neither can provide the whole answer to improve yields and soil health.
So the SPot Farm project combines research trials and demonstrations with industry best practice at exemplars such as the 22,500

In [24]:
start_tokens = inps["input_ids"].shape[1]
total_tokens = outputs_tokens.shape[1]

print(f"total_tokens = {total_tokens}")
print(f"new_tokens = {total_tokens - start_tokens}")
print(f"num_seconds = {end - start}")
print(f"new tok/sec = {(total_tokens - start_tokens) / (end - start)}")

total_tokens = 1266
new_tokens = 213
num_seconds = 10.281893671490252
new tok/sec = 20.7160282731389


In [2]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch

# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype=torch.float16,
#     device_map={'':0}
# )

# tokenizer = AutoTokenizer.from_pretrained(model_id)

/home/rshaw/zephyr-training/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:18<00:00,  9.06s/it]


In [1]:
from fastchat.model import load_model, get_conversation_template
import torch

# model_path = "/home/rshaw/zephyr-training/pruning/data/zephyr-50sparse-fp16-one-shot-v0"
model_path = "HuggingFaceH4/zephyr-7b-beta"
model_id = "zephyr-beta"

model, tokenizer = load_model(
    model_path,
    device="cuda",
    num_gpus=1,
    dtype=torch.float16,
    load_8bit=False,
    cpu_offloading=False,
    debug=False,
)

/home/rshaw/zephyr-training/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading checkpoint shards: 100%|██████████| 8/8 [00:27<00:00,  3.41s/it]


In [4]:
from FastChat.fastchat.llm_judge.common import load_questions, temperature_config

question_file = "FastChat/fastchat/llm_judge/data/mt_bench/question.jsonl"
questions = load_questions(question_file, None, None)
question = questions[2]
print(question)

conv = get_conversation_template(model_id)

turns = []
for j in range(len(question["turns"])):
    qs = question["turns"][j]
    conv.append_message(conv.roles[0], qs)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()

    do_sample=True
    temperature=0.7
    print(prompt)
    
    input_ids = tokenizer([prompt]).input_ids

    output_ids = model.generate(
        torch.as_tensor(input_ids).cuda(),
        do_sample=do_sample,
        temperature=temperature,
        max_new_tokens=1024,
        use_cache=True
    )

    break

{'question_id': 83, 'category': 'writing', 'turns': ['Imagine you are writing a blog post comparing two popular smartphone models. Develop an outline for the blog post, including key points and subheadings to effectively compare and contrast the features, performance, and user experience of the two models. Please answer in fewer than 200 words.', 'Take your previous response and rephrase it as a limerick.']}
<|system|>
</s>
<|user|>
Imagine you are writing a blog post comparing two popular smartphone models. Develop an outline for the blog post, including key points and subheadings to effectively compare and contrast the features, performance, and user experience of the two models. Please answer in fewer than 200 words.</s>
<|assistant|>



In [6]:
print(tokenizer.batch_decode(output_ids)[0])

<s> <|system|>
</s> 
<|user|>
Imagine you are writing a blog post comparing two popular smartphone models. Develop an outline for the blog post, including key points and subheadings to effectively compare and contrast the features, performance, and user experience of the two models. Please answer in fewer than 200 words.</s> 
<|assistant|>
Title: Smartphone Showdown: iPhone 12 vs. Samsung Galaxy S21

Introduction:

Smartphones have become a vital part of our daily lives, and with new models coming out every year, it can be overwhelming to decide which one to pick. In this blog post, we will compare two popular smartphone models, the iPhone 12, and Samsung Galaxy S21. We will take a closer look at their features, performance, and user experience to help you make an informed decision.

1. Design and Display

- iPhone 12: The iPhone 12 comes with a new design, featuring a flat-edged rectangular shape. It has a 6.1-inch Super Retina XDR OLED display with a 2532 x 1170 resolution. The displ